<a href="https://colab.research.google.com/github/li0217codeninja/time-seq-learning/blob/main/timeseriesTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python --version

Python 3.10.12


In [10]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import numpy as np

Timeseries PyTorch Transformer

In [ ]:
class SequenceDataset(torch.utils.data.Dataset):
    def __init__(self, seq_length):
        self.seq_length = seq_length

    def __len__(self):
        return 10000  # Replace with actual dataset size

    def __getitem__(self, idx):
        # Generate a sequence number
        seq_num = torch.tensor(idx)

        # Generate a sequence of numbers based on seq_num
        seq_data = torch.randn(self.seq_length) * seq_num

        return seq_num, seq_data

def collate_fn(samples):
    seq_nums, seq_data = zip(*samples)
    # Pad or truncate sequences to a fixed length (optional)
    # ...
    return torch.stack(seq_nums), torch.stack(seq_data)

train_loader = DataLoader(
    SequenceDataset(seq_length=32),
    batch_size=32,
    shuffle=True,
    collate_fn=collate_fn,
)


In [ ]:
transformer_model = nn.Transformer(nhead=16)
src = torch.rand(10, 32, 512)
tgt = torch.rand(20, 32, 512)
out = transformer_model(src, tgt)

In [ ]:
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(transformer_model.parameters())

epochs = 10
for epoch in range(epochs):
  for batch_idx, (data, target) in enumerate(train_loader):
    # Forward pass
    output = transformer_model(data,target)
    # Loss calculation
    loss = loss_fn(output, target)
    # Backward pass and parameter update
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


In [ ]:
transformer_model.parameters()

<generator object Module.parameters at 0x7a4ca5ac67a0>

## Custom Train loop with LSTM



In [11]:
class RNNmodel(nn.Module):
  def __init__(self, input_size, hidden_size,  num_features):
    super(RNNmodel,self).__init__() #todo check super()
    self.lstm = nn.LSTM(input_size, hidden_size,  batch_first=True)
    self.dense = nn.Linear(hidden_size, num_features)

  def forward(self, x):
    h, _ = self.lstm(x)
    output = self.dense(h[:,-1,:])
    return output


In [16]:
class DummyDataset(Dataset):
  def __init__(self,  data, sequence_len):
    # 100 sequences, 10 time steps, 2 features per variable, 3 variables
    self.data = data
    self.sequence_length = sequence_len
  def __len__(self):
    return len(self.data) - self.sequence_length

  def __getitem__(self, idx):
        x = torch.tensor(self.data[idx:idx + self.sequence_length, :], dtype=torch.float32)
        y = torch.tensor(self.data[idx + self.sequence_length, :], dtype=torch.float32)
        return x, y



In [23]:
# Generate dummy data
np.random.seed(42)
num_samples = 1000
sequence_length = 10
num_features = 3

data = np.random.randn(num_samples, num_features)

# Create the model, dataset, and data loader
input_size = num_features
hidden_size = 50
output_size = num_features

model = RNNmodel(input_size, hidden_size, output_size)
dataset = DummyDataset(data, sequence_length)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, drop_last=True)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# Example usage:
for epoch in range(10):
  for batch_x, batch_y in dataloader:
        # Training logic goes here
        # e.g., forward pass, loss computation, backward pass, and optimization
        outputs = model(batch_x)
        loss = nn.MSELoss()(outputs, batch_y)

        # Your training steps here
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

  print(f'Epoch {epoch + 1}, Batch Loss: {loss.item()}')


# After training, you can use the trained model for predictions
# For example, if you have a test sequence 'test_sequence':
# test_sequence = torch.tensor(test_sequence, dtype=torch.float32).unsqueeze(0)  # Add batch dimension
# prediction = model(test_sequence)

Epoch 1, Batch Loss: 1.082436203956604
Epoch 2, Batch Loss: 0.8982471823692322
Epoch 3, Batch Loss: 0.8221070170402527
Epoch 4, Batch Loss: 0.8753069043159485
Epoch 5, Batch Loss: 1.306394100189209
Epoch 6, Batch Loss: 0.7554345726966858
Epoch 7, Batch Loss: 0.8944863677024841
Epoch 8, Batch Loss: 0.7750611901283264
Epoch 9, Batch Loss: 0.9092220664024353
Epoch 10, Batch Loss: 0.7927865982055664
